## Deliverable E, Part A: Preprocessing (Group 20)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
import pandas.io.sql as sqlio
import psycopg2
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

pd.set_option('display.max_columns', None)

In [3]:
f = open(r"C:\Users\16136\Documents\passwordpy.txt", "r")
password = f.readline()
conn = psycopg2.connect(
    host = "www.eecs.uottawa.ca",
    port = 15432,
    user = "cbrad077",
    database = "group_20",
    password = password
)

In [4]:
features_query  = '''SELECT r.holiday, r.weekend, p.age, p.gender, p.acquisition_group, p.outbreak_related, s.begin_date, s.end_date, s.city, s.zone_measures,  s.lockdown, s.stay_at_home_order, s.max_indoor_capacity, s.max_outdoor_capacity, d.*, w.*
FROM fact_table f, d_patient p, d_special_measures s, d_mobility d, d_weather w, d_reported_date r
WHERE f.patient_key = p.patient_surrogate_key AND f.special_measures_key = s.surrogate_key AND f.mobility_key = d.mobility_key AND f.weather_key = w.weather_key AND f.reported_date_key = r.reported_date_surrogate_key
'''

features_data = sqlio.read_sql_query(features_query, conn)

conn = None

In [5]:
features_data.head()

,holiday,weekend,age,gender,acquisition_group,outbreak_related,begin_date,end_date,city,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,mobility_key,subregion,province,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,weather_key,daily_high_temp,daily_low_temp,total_precipitation,station_name,daily_mean,weather_date
0,False,False,40s,FEMALE,OB,True,2020-11-23,2020-12-26,Toronto,Grey,1,0,0,10,181,Toronto Division,Ontario,-32.0,19.0,1.0,-60.0,-55.0,18.0,180,6.3,0.5,1.8,TORONTO INTL A,3.4,2020-12-23
1,False,False,80s,FEMALE,NO KNOWN EPI LINK,False,2020-11-14,2020-11-23,Toronto,Red,0,0,10,25,145,Toronto Division,Ontario,-41.0,-13.0,-22.0,-58.0,-48.0,18.0,144,3.0,-5.4,0.6,TORONTO INTL A,-1.2,2020-11-17
2,False,False,30s,MALE,CC,False,2020-10-27,2020-11-14,Toronto,Orange,0,0,50,100,127,Toronto Division,Ontario,-37.0,-6.0,-10.0,-53.0,-45.0,17.0,126,2.3,-4.3,0.0,TORONTO INTL A,-1.0,2020-10-30
3,True,False,20s,FEMALE,NO KNOWN EPI LINK,False,2020-10-27,2020-11-14,Toronto,Orange,0,0,50,100,139,Toronto Division,Ontario,-35.0,-9.0,25.0,-56.0,-48.0,18.0,138,18.2,5.5,0.6,TORONTO INTL A,11.9,2020-11-11
4,False,False,50s,MALE,NO KNOWN EPI LINK,False,2020-12-26,2021-01-12,Toronto,Grey,1,1,0,10,196,Toronto Division,Ontario,-53.0,-15.0,-12.0,-66.0,-54.0,24.0,195,-0.6,-7.1,0.0,TORONTO INTL A,-3.9,2021-01-07


In [6]:
# create pandas dataframe

features = pd.DataFrame(features_data, columns=["holiday", "weekend", "city", "daily_mean", 
                                                "total_precipitation", "zone_measures", "lockdown", "stay_at_home_order", 
                                                "max_indoor_capacity", "max_outdoor_capacity", "retail_and_recreation",
                                                "grocery_and_pharmacy", "parks", "transit_stations", "workplaces", 
                                                "residential", "age", "gender", "acquisition_group", "outbreak_related"])

features

,holiday,weekend,city,daily_mean,total_precipitation,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,acquisition_group,outbreak_related
0,False,False,Toronto,3.4,1.8,Grey,1,0,0,10,-32.0,19.0,1.0,-60.0,-55.0,18.0,40s,FEMALE,OB,True
1,False,False,Toronto,-1.2,0.6,Red,0,0,10,25,-41.0,-13.0,-22.0,-58.0,-48.0,18.0,80s,FEMALE,NO KNOWN EPI LINK,False
2,False,False,Toronto,-1.0,0.0,Orange,0,0,50,100,-37.0,-6.0,-10.0,-53.0,-45.0,17.0,30s,MALE,CC,False
3,True,False,Toronto,11.9,0.6,Orange,0,0,50,100,-35.0,-9.0,25.0,-56.0,-48.0,18.0,20s,FEMALE,NO KNOWN EPI LINK,False
4,False,False,Toronto,-3.9,0.0,Grey,1,1,0,10,-53.0,-15.0,-12.0,-66.0,-54.0,24.0,50s,MALE,NO KNOWN EPI LINK,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56245,False,False,Toronto,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,20s,MALE,CC,False
56246,False,False,Toronto,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,40s,FEMALE,NO KNOWN EPI LINK,True
56247,False,False,Toronto,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,<20,FEMALE,CC,False
56248,False,False,Toronto,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,<20,MALE,CC,False


In [7]:
# check for missing values 

features.isna().sum()

holiday                  0
weekend                  0
city                     0
daily_mean               0
total_precipitation      0
zone_measures            0
lockdown                 0
stay_at_home_order       0
max_indoor_capacity      0
max_outdoor_capacity     0
retail_and_recreation    0
grocery_and_pharmacy     0
parks                    0
transit_stations         0
workplaces               0
residential              0
age                      0
gender                   0
acquisition_group        0
outbreak_related         0
dtype: int64

In [8]:
# view value distribution

Counter(features["outbreak_related"])

Counter({True: 10964, False: 45286})

In [9]:
Counter(features["gender"])

Counter({'FEMALE': 28655,
         'MALE': 27215,
         'UNSPECIFIED': 365,
         'GENDER DIVERSE': 15})

In [10]:
Counter(features["city"])

Counter({'Toronto': 49437, 'Ottawa': 6813})

In [11]:
drop_index = features[features["gender"] == "UNSPECIFIED"].index
features.drop(drop_index, inplace = True)
drop_index = features[features["gender"] == "GENDER DIVERSE"].index
features.drop(drop_index, inplace = True)
drop_index = features[features["age"] == "UNKNOWN"].index
features.drop(drop_index, inplace = True)
features

,holiday,weekend,city,daily_mean,total_precipitation,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,acquisition_group,outbreak_related
0,False,False,Toronto,3.4,1.8,Grey,1,0,0,10,-32.0,19.0,1.0,-60.0,-55.0,18.0,40s,FEMALE,OB,True
1,False,False,Toronto,-1.2,0.6,Red,0,0,10,25,-41.0,-13.0,-22.0,-58.0,-48.0,18.0,80s,FEMALE,NO KNOWN EPI LINK,False
2,False,False,Toronto,-1.0,0.0,Orange,0,0,50,100,-37.0,-6.0,-10.0,-53.0,-45.0,17.0,30s,MALE,CC,False
3,True,False,Toronto,11.9,0.6,Orange,0,0,50,100,-35.0,-9.0,25.0,-56.0,-48.0,18.0,20s,FEMALE,NO KNOWN EPI LINK,False
4,False,False,Toronto,-3.9,0.0,Grey,1,1,0,10,-53.0,-15.0,-12.0,-66.0,-54.0,24.0,50s,MALE,NO KNOWN EPI LINK,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56245,False,False,Toronto,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,20s,MALE,CC,False
56246,False,False,Toronto,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,40s,FEMALE,NO KNOWN EPI LINK,True
56247,False,False,Toronto,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,<20,FEMALE,CC,False
56248,False,False,Toronto,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,<20,MALE,CC,False


In [12]:
# convert binary columns to 1/0 

features["outbreak_related"].replace({True: 1, False: 0}, inplace=True)
features["gender"].replace({"FEMALE": 1,"MALE": 0}, inplace=True)
features["city"].replace({"Ottawa": 1, "Toronto": 0}, inplace=True)
features["holiday"].replace({True: 1, False: 0}, inplace=True)
features["weekend"].replace({True: 1, False: 0}, inplace=True)

features

,holiday,weekend,city,daily_mean,total_precipitation,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,acquisition_group,outbreak_related
0,0,0,0,3.4,1.8,Grey,1,0,0,10,-32.0,19.0,1.0,-60.0,-55.0,18.0,40s,1,OB,1
1,0,0,0,-1.2,0.6,Red,0,0,10,25,-41.0,-13.0,-22.0,-58.0,-48.0,18.0,80s,1,NO KNOWN EPI LINK,0
2,0,0,0,-1.0,0.0,Orange,0,0,50,100,-37.0,-6.0,-10.0,-53.0,-45.0,17.0,30s,0,CC,0
3,1,0,0,11.9,0.6,Orange,0,0,50,100,-35.0,-9.0,25.0,-56.0,-48.0,18.0,20s,1,NO KNOWN EPI LINK,0
4,0,0,0,-3.9,0.0,Grey,1,1,0,10,-53.0,-15.0,-12.0,-66.0,-54.0,24.0,50s,0,NO KNOWN EPI LINK,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56245,0,0,0,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,20s,0,CC,0
56246,0,0,0,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,40s,1,NO KNOWN EPI LINK,1
56247,0,0,0,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,<20,1,CC,0
56248,0,0,0,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,<20,0,CC,0


In [13]:
# fix age 

Counter(features["age"])

Counter({'40s': 8161,
         '80s': 2415,
         '30s': 8999,
         '20s': 10390,
         '50s': 8196,
         '<20': 7466,
         '70s': 3150,
         '60s': 5671,
         '90+': 1412})

In [14]:
# convert ages to normalized ordinals

features["age"].replace({'<20': 0.1, '20s': 0.2, '30s': 0.3, '40s': 0.4, 
                              '50s': 0.5, '60s': 0.6, '70s': 0.7, '80s': 0.8, '90+': 0.9}, inplace=True)
features

,holiday,weekend,city,daily_mean,total_precipitation,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,acquisition_group,outbreak_related
0,0,0,0,3.4,1.8,Grey,1,0,0,10,-32.0,19.0,1.0,-60.0,-55.0,18.0,0.4,1,OB,1
1,0,0,0,-1.2,0.6,Red,0,0,10,25,-41.0,-13.0,-22.0,-58.0,-48.0,18.0,0.8,1,NO KNOWN EPI LINK,0
2,0,0,0,-1.0,0.0,Orange,0,0,50,100,-37.0,-6.0,-10.0,-53.0,-45.0,17.0,0.3,0,CC,0
3,1,0,0,11.9,0.6,Orange,0,0,50,100,-35.0,-9.0,25.0,-56.0,-48.0,18.0,0.2,1,NO KNOWN EPI LINK,0
4,0,0,0,-3.9,0.0,Grey,1,1,0,10,-53.0,-15.0,-12.0,-66.0,-54.0,24.0,0.5,0,NO KNOWN EPI LINK,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56245,0,0,0,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,0.2,0,CC,0
56246,0,0,0,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,0.4,1,NO KNOWN EPI LINK,1
56247,0,0,0,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,0.1,1,CC,0
56248,0,0,0,-1.6,0.0,Red,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,0.1,0,CC,0


In [15]:
# one-hot-encoding columns 

zone_encoding = pd.get_dummies(features["zone_measures"])
acq_encoding = pd.get_dummies(features["acquisition_group"])

# merge with original set 

features = pd.merge(features, acq_encoding, left_index=True, right_index=True)
features = pd.merge(features, zone_encoding, left_index=True, right_index=True)

# drop old columns 

del features["zone_measures"]
del features["acquisition_group"]

features

,holiday,weekend,city,daily_mean,total_precipitation,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,outbreak_related,CC,MISSING INFORMATION,NO KNOWN EPI LINK,OB,TRAVEL,UNSPECIFIED EPI LINK,Grey,Orange,Red
0,0,0,0,3.4,1.8,1,0,0,10,-32.0,19.0,1.0,-60.0,-55.0,18.0,0.4,1,1,0,0,0,1,0,0,1,0,0
1,0,0,0,-1.2,0.6,0,0,10,25,-41.0,-13.0,-22.0,-58.0,-48.0,18.0,0.8,1,0,0,0,1,0,0,0,0,0,1
2,0,0,0,-1.0,0.0,0,0,50,100,-37.0,-6.0,-10.0,-53.0,-45.0,17.0,0.3,0,0,1,0,0,0,0,0,0,1,0
3,1,0,0,11.9,0.6,0,0,50,100,-35.0,-9.0,25.0,-56.0,-48.0,18.0,0.2,1,0,0,0,1,0,0,0,0,1,0
4,0,0,0,-3.9,0.0,1,1,0,10,-53.0,-15.0,-12.0,-66.0,-54.0,24.0,0.5,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56245,0,0,0,-1.6,0.0,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,0.2,0,0,1,0,0,0,0,0,0,0,1
56246,0,0,0,-1.6,0.0,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,0.4,1,1,0,0,1,0,0,0,0,0,1
56247,0,0,0,-1.6,0.0,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,0.1,1,0,1,0,0,0,0,0,0,0,1
56248,0,0,0,-1.6,0.0,0,0,10,25,-50.0,-14.0,-6.0,-63.0,-49.0,21.0,0.1,0,0,1,0,0,0,0,0,0,0,1


In [16]:
# normalize all numerical columns using min-max scaling 

scaler = MinMaxScaler()

features[["daily_mean", "total_precipitation", "max_indoor_capacity", "max_outdoor_capacity", 
          "retail_and_recreation", "grocery_and_pharmacy", "parks", "transit_stations", "workplaces", 
          "residential"]] = scaler.fit_transform(features[["daily_mean", "total_precipitation", 
                                                                         "max_indoor_capacity","max_outdoor_capacity",
                                                                         "retail_and_recreation", "grocery_and_pharmacy", 
                                                                         "parks", "transit_stations", "workplaces", "residential"]])

features

,holiday,weekend,city,daily_mean,total_precipitation,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,outbreak_related,CC,MISSING INFORMATION,NO KNOWN EPI LINK,OB,TRAVEL,UNSPECIFIED EPI LINK,Grey,Orange,Red
0,0,0,0,0.636364,0.070039,1,0,0.0,0.000000,0.7000,0.829268,0.271795,0.500000,0.352273,0.483871,0.4,1,1,0,0,0,1,0,0,1,0,0
1,0,0,0,0.505682,0.023346,0,0,0.2,0.166667,0.5875,0.569106,0.153846,0.537037,0.431818,0.483871,0.8,1,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0.511364,0.000000,0,0,1.0,1.000000,0.6375,0.626016,0.215385,0.629630,0.465909,0.451613,0.3,0,0,1,0,0,0,0,0,0,1,0
3,1,0,0,0.877841,0.023346,0,0,1.0,1.000000,0.6625,0.601626,0.394872,0.574074,0.431818,0.483871,0.2,1,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0.428977,0.000000,1,1,0.0,0.000000,0.4375,0.552846,0.205128,0.388889,0.363636,0.677419,0.5,0,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56245,0,0,0,0.494318,0.000000,0,0,0.2,0.166667,0.4750,0.560976,0.235897,0.444444,0.420455,0.580645,0.2,0,0,1,0,0,0,0,0,0,0,1
56246,0,0,0,0.494318,0.000000,0,0,0.2,0.166667,0.4750,0.560976,0.235897,0.444444,0.420455,0.580645,0.4,1,1,0,0,1,0,0,0,0,0,1
56247,0,0,0,0.494318,0.000000,0,0,0.2,0.166667,0.4750,0.560976,0.235897,0.444444,0.420455,0.580645,0.1,1,0,1,0,0,0,0,0,0,0,1
56248,0,0,0,0.494318,0.000000,0,0,0.2,0.166667,0.4750,0.560976,0.235897,0.444444,0.420455,0.580645,0.1,0,0,1,0,0,0,0,0,0,0,1


In [17]:
# get labels, remove from features set

labels = features["outbreak_related"]
del features["outbreak_related"]

labels

0        1
1        0
2        0
3        0
4        0
        ..
56245    0
56246    1
56247    0
56248    0
56249    0
Name: outbreak_related, Length: 55860, dtype: int64

In [18]:
# get feature values 

feature_values = features.values

feature_values

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [19]:
# initial number of features

len(feature_values[1])

26

In [20]:
# feature selection (low variance method)

selector = VarianceThreshold(threshold=(.8*(1-.8)))
selector.fit_transform(feature_values)

array([[0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
# number of features after feature selection

len(feature_values[1])

26

In [22]:
# try another method of feature selection (tree-based)

feature_values.shape

(55860, 26)

In [23]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(feature_values, labels)
model = SelectFromModel(clf, prefit=True)
feature_idx = model.get_support()
feature_name = features.columns[feature_idx]
feature_values_new = model.transform(feature_values)
feature_values_new.shape  

(55860, 5)

In [24]:
feature_values_new

array([[0.4, 0. , 0. , 0. , 1. ],
       [0.8, 0. , 0. , 1. , 0. ],
       [0.3, 1. , 0. , 0. , 0. ],
       ...,
       [0.1, 1. , 0. , 0. , 0. ],
       [0.1, 1. , 0. , 0. , 0. ],
       [0.2, 1. , 0. , 0. , 0. ]])

In [25]:
feature_name

Index(['age', 'CC', 'MISSING INFORMATION', 'NO KNOWN EPI LINK', 'OB'], dtype='object')

In [26]:
# split into train and test sets (stratified)

x_train, x_test, y_train, y_test = train_test_split(feature_values, labels, test_size = 0.2, shuffle = True, stratify = labels)

In [27]:
Counter(y_train)

Counter({0: 36002, 1: 8686})

In [28]:
Counter(y_test)

Counter({0: 9001, 1: 2171})

In [31]:
# export finished data sets 

features.to_csv("Features.csv",index=False)
labels.to_csv("Labels.csv",index=False)
np.savetxt("Feature_Values.csv", feature_values, delimiter=",")
np.savetxt("Feature_Values_Selected.csv", feature_values_new, delimiter=",")
np.savetxt("X_Train.csv", x_train, delimiter=",")
np.savetxt("X_Test.csv", x_test, delimiter=",")
np.savetxt("Y_Train.csv", y_train, delimiter=",")
np.savetxt("Y_Test.csv", y_test, delimiter=",")